In [1]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

In [4]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import os
from PIL import ImageFile

# Skip truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Clear GPU memory
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define dataset paths
data_dir = "/home/vu-lab03-pc26/Downloads/Merged_Dataset"
train_dir = f"{data_dir}/train"
val_dir = f"{data_dir}/validation"
test_dir = f"{data_dir}/test"

# Define transformations
transform = transforms.Compose([
    transforms.Resize((380, 380)),  
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2, pin_memory=True)

# Load Pre-trained EfficientNet-B4
model = EfficientNet.from_pretrained("efficientnet-b4")
num_features = model._fc.in_features
model._fc = nn.Linear(num_features, 2)  # Binary classification
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Enable mixed precision
scaler = torch.cuda.amp.GradScaler()

# Train Model with Batch Tracking
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

    for epoch in range(epochs):
        # TRAINING PHASE
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        tqdm_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}")

        optimizer.zero_grad()
        for i, (inputs, labels) in tqdm_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels) / 4  # Gradient accumulation
            
            scaler.scale(loss).backward()
            
            if (i + 1) % 4 == 0:  # Update every 4 steps
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            running_loss += loss.item() * 4
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            # Update progress bar with live batch accuracy & loss
            batch_acc = 100 * correct_train / total_train
            tqdm_bar.set_postfix(loss=running_loss / (i + 1), acc=batch_acc)

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct_train / total_train

        # VALIDATION PHASE
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_loss = running_val_loss / len(val_loader)
        val_acc = 100 * correct_val / total_val

        # Store values in history
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        # Print epoch summary
        print(f"\nEpoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%\n")

    return history

# Train the model
history = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)

# Save the model
torch.save(model.state_dict(), "efficientnet_deepfake_classifier.pth")

Using device: cuda
Loaded pretrained weights for efficientnet-b4


/tmp/ipykernel_5650/1834900099.py:57: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1/10:   0%|                                     | 0/30645 [00:00<?, ?it/s]/tmp/ipykernel_5650/1834900099.py:76: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/10: 100%|████| 30645/30645 [48:05<00:00, 10.62it/s, acc=97, loss=0.0772]



Epoch 1/10 - Train Loss: 0.0772, Train Acc: 97.02% | Val Loss: 0.0555, Val Acc: 97.90%



Epoch 2/10: 100%|██| 30645/30645 [22:16<00:00, 22.93it/s, acc=98.7, loss=0.0338]



Epoch 2/10 - Train Loss: 0.0338, Train Acc: 98.68% | Val Loss: 0.0323, Val Acc: 98.90%



...


Epoch 8/10 - Train Loss: 0.0126, Train Acc: 99.51% | Val Loss: 0.0593, Val Acc: 98.71%



Epoch 9/10: 100%|██| 30645/30645 [23:47<00:00, 21.47it/s, acc=99.6, loss=0.0115]



Epoch 9/10 - Train Loss: 0.0115, Train Acc: 99.57% | Val Loss: 0.0275, Val Acc: 99.14%



Epoch 10/10: 100%|█| 30645/30645 [23:19<00:00, 21.90it/s, acc=99.6, loss=0.0103]



Epoch 10/10 - Train Loss: 0.0103, Train Acc: 99.62% | Val Loss: 0.0615, Val Acc: 98.98%



In [5]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import os
import numpy as np

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define test dataset path
data_dir = "/home/vu-lab03-pc26/Downloads/Merged_Dataset"
test_dir = f"{data_dir}/test"

# Define the same transformations used during training
transform = transforms.Compose([
    transforms.Resize((380, 380)),  
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2, pin_memory=True)

# Load the trained model
model = EfficientNet.from_name("efficientnet-b4")  # Initialize model structure
num_features = model._fc.in_features
model._fc = nn.Linear(num_features, 2)  # Binary classification (2 classes)
model.load_state_dict(torch.load("efficientnet_deepfake_classifier.pth", map_location=device))  # Load trained weights
model = model.to(device)
model.eval()  # Set model to evaluation mode

# Define loss function
criterion = nn.CrossEntropyLoss()

# Initialize variables for evaluation
correct = 0
total = 0
all_preds = []
all_labels = []
test_loss = 0.0

# Disable gradient calculations for evaluation
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate test accuracy
test_accuracy = 100 * correct / total
avg_test_loss = test_loss / len(test_loader)

# Compute confusion matrix and classification report
conf_matrix = confusion_matrix(all_labels, all_preds)
class_report = classification_report(all_labels, all_preds, target_names=test_dataset.classes)

# Print the results
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(f"Test Loss: {avg_test_loss:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


Using device: cuda
Test Accuracy: 96.16%
Test Loss: 0.2169

Confusion Matrix:
[[15321   171]
 [ 1015 14398]]

Classification Report:
              precision    recall  f1-score   support

        Fake       0.94      0.99      0.96     15492
        Real       0.99      0.93      0.96     15413

    accuracy                           0.96     30905
   macro avg       0.96      0.96      0.96     30905
weighted avg       0.96      0.96      0.96     30905



In [13]:
import torch
import torch.nn as nn
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from PIL import Image
import os

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the same image transformations used during training
transform = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the trained model
model = EfficientNet.from_name("efficientnet-b4")
num_features = model._fc.in_features
model._fc = nn.Linear(num_features, 2)  # Binary classification

# Load trained weights
model.load_state_dict(torch.load("efficientnet_deepfake_classifier.pth", map_location=device))
model.to(device)
model.eval()  # Set the model to evaluation mode

# Define class labels
class_labels = {0: "Fake", 1: "Real"}

# Function to predict if an image is fake or real
def predict_image(image_path):
    if not os.path.exists(image_path):
        print(f"Error: File '{image_path}' not found.")
        return
    
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    # Make prediction
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        confidence = torch.nn.functional.softmax(output, dim=1)[0][predicted].item() * 100

    # Print result
    print(f"Prediction: {class_labels[predicted.item()]} (Confidence: {confidence:.2f}%)")

# Test with an image
image_path = "/home/vu-lab03-pc26/Downloads/Untitled.jpeg"  # Change this to the path of your test image
predict_image(image_path)


Using device: cuda
Prediction: Real (Confidence: 99.97%)
